In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('order_history_kaggle_data.csv')

In [4]:
df.head(5)

,Restaurant ID,Restaurant name,Subzone,City,Order ID,Order Placed At,Order Status,Delivery,Distance,Items in order,...,Rating,Review,Cancellation / Rejection reason,Restaurant compensation (Cancellation),Restaurant penalty (Rejection),KPT duration (minutes),Rider wait time (minutes),Order Ready Marked,Customer complaint tag,Customer ID
0,20320607,Swaad,Sector 4,Delhi NCR,6168884918,"11:38 PM, September 10 2024",Delivered,Zomato Delivery,3km,"1 x Grilled Chicken Jamaican Tender, 1 x Grill...",...,NaN,NaN,NaN,NaN,NaN,18.35,11.6,Correctly,NaN,5d6c2b96db963098bc69768bea504c8bf46106a8a5178e...
1,20320607,Swaad,Sector 4,Delhi NCR,6170707559,"11:34 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Peri Peri Fries, 1 x Fried Chicken Angara ...",...,NaN,NaN,NaN,NaN,NaN,16.95,3.6,Correctly,NaN,0781815deb4a10a574e9fee4fa0b86b074d4a0b36175d5...
2,20320607,Swaad,Sector 4,Delhi NCR,6169375019,"03:52 PM, September 10 2024",Delivered,Zomato Delivery,<1km,1 x Bone in Peri Peri Grilled Chicken,...,NaN,NaN,NaN,NaN,NaN,14.05,12.2,Correctly,NaN,f93362f5ce5382657482d164e368186bcec9c6225fd93d...
3,20320607,Swaad,Sector 4,Delhi NCR,6151677434,"03:45 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Fried Chicken Ghostbuster Tender, 1 x Anga...",...,4.0,NaN,NaN,NaN,NaN,19.00,3.3,Correctly,NaN,1ed226d1b8a5f7acee12fc1d6676558330a3b2b742af5d...
4,20320607,Swaad,Sector 4,Delhi NCR,6167540897,"03:04 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Peri Peri Krispers, 1 x Fried Chicken Anga...",...,NaN,NaN,NaN,NaN,NaN,15.97,1.0,Correctly,NaN,d21a2ac6ea06b31cc3288ab20c4ef2f292066c096f2c5f...


In [5]:
df.isna().sum()

Restaurant ID                                             0
Restaurant name                                           0
Subzone                                                   0
City                                                      0
Order ID                                                  0
Order Placed At                                           0
Order Status                                              0
Delivery                                                  0
Distance                                                  0
Items in order                                            0
Instructions                                          20601
Discount construct                                     5498
Bill subtotal                                             0
Packaging charges                                         0
Restaurant discount (Promo)                               0
Restaurant discount (Flat offs, Freebies & others)        0
Gold discount                           

In [6]:
df.describe()

,Restaurant ID,Order ID,Bill subtotal,Packaging charges,Restaurant discount (Promo),"Restaurant discount (Flat offs, Freebies & others)",Gold discount,Brand pack discount,Total,Rating,Restaurant compensation (Cancellation),Restaurant penalty (Rejection),KPT duration (minutes),Rider wait time (minutes)
count,2.132100e+04,2.132100e+04,21321.000000,21321.000000,21321.000000,21321.000000,21321.000000,21321.000000,21321.000000,2491.000000,133.000000,3.0,21026.000000,21153.000000
mean,2.074413e+07,6.354622e+09,750.076838,32.564592,65.091816,31.795058,0.099128,3.039324,682.616113,4.356885,356.409549,0.0,17.332960,4.825070
std,2.447193e+05,1.230263e+08,498.759428,22.235898,85.401604,131.487091,3.264261,17.070780,465.313977,1.181472,328.128240,0.0,6.283388,4.982591
min,2.032061e+07,6.086767e+09,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.500000,1.000000,83.580000,0.0,0.000000,0.100000
25%,2.063570e+07,6.250751e+09,459.000000,18.450000,0.000000,0.000000,0.000000,0.000000,387.450000,4.000000,191.950000,0.0,13.380000,1.000000
50%,2.065987e+07,6.357715e+09,629.000000,28.450000,80.000000,0.000000,0.000000,0.000000,597.450000,5.000000,272.580000,0.0,16.330000,3.100000
75%,2.088265e+07,6.456827e+09,899.000000,39.950000,100.000000,0.000000,0.000000,0.000000,837.900000,5.000000,397.840000,0.0,20.050000,7.400000
max,2.152306e+07,6.573392e+09,16080.000000,603.000000,4020.000000,7787.000000,280.100000,554.800000,12663.000000,5.000000,3236.980000,0.0,90.870000,73.800000


In [7]:
df.info

<bound method DataFrame.info of        Restaurant ID       Restaurant name            Subzone       City  \
0           20320607                 Swaad           Sector 4  Delhi NCR   
1           20320607                 Swaad           Sector 4  Delhi NCR   
2           20320607                 Swaad           Sector 4  Delhi NCR   
3           20320607                 Swaad           Sector 4  Delhi NCR   
4           20320607                 Swaad           Sector 4  Delhi NCR   
...              ...                   ...                ...        ...   
21316       21467440  The Chicken Junction  Chittaranjan Park  Delhi NCR   
21317       21467440  The Chicken Junction  Chittaranjan Park  Delhi NCR   
21318       21523055       Masala Junction           Sector 4  Delhi NCR   
21319       21523055       Masala Junction           Sector 4  Delhi NCR   
21320       21523055       Masala Junction           Sector 4  Delhi NCR   

         Order ID              Order Placed At Order St

In [8]:
df.columns

Index(['Restaurant ID', 'Restaurant name', 'Subzone', 'City', 'Order ID',
       'Order Placed At', 'Order Status', 'Delivery', 'Distance',
       'Items in order', 'Instructions', 'Discount construct', 'Bill subtotal',
       'Packaging charges', 'Restaurant discount (Promo)',
       'Restaurant discount (Flat offs, Freebies & others)', 'Gold discount',
       'Brand pack discount', 'Total', 'Rating', 'Review',
       'Cancellation / Rejection reason',
       'Restaurant compensation (Cancellation)',
       'Restaurant penalty (Rejection)', 'KPT duration (minutes)',
       'Rider wait time (minutes)', 'Order Ready Marked',
       'Customer complaint tag', 'Customer ID'],
      dtype='object')

In [9]:
features = [
    'Delivery',
    'Distance',
    'Items in order',
    'Bill subtotal',
    'Total',
    'KPT duration (minutes)',
    'Rider wait time (minutes)',
    'Subzone',
    'City'
]

In [14]:
df[features].head()

,Delivery,Distance,Items in order,Bill subtotal,Total,KPT duration (minutes),Rider wait time (minutes),Subzone,City
0,Zomato Delivery,3km,"1 x Grilled Chicken Jamaican Tender, 1 x Grill...",715.0,666.75,18.35,11.6,Sector 4,Delhi NCR
1,Zomato Delivery,2km,"1 x Peri Peri Fries, 1 x Fried Chicken Angara ...",1179.0,1054.20,16.95,3.6,Sector 4,Delhi NCR
2,Zomato Delivery,<1km,1 x Bone in Peri Peri Grilled Chicken,310.0,241.50,14.05,12.2,Sector 4,Delhi NCR
3,Zomato Delivery,2km,"1 x Fried Chicken Ghostbuster Tender, 1 x Anga...",620.0,567.00,19.00,3.3,Sector 4,Delhi NCR
4,Zomato Delivery,2km,"1 x Peri Peri Krispers, 1 x Fried Chicken Anga...",584.0,529.20,15.97,1.0,Sector 4,Delhi NCR


In [10]:
df['high_rating'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)

In [15]:
df['high_rating']

0        0
1        0
2        0
3        1
4        0
        ..
21316    0
21317    0
21318    1
21319    1
21320    0
Name: high_rating, Length: 21321, dtype: int64

In [16]:
df = df.dropna(subset=['Rating'])

In [22]:
df['KPT duration (minutes)'].fillna(df['KPT duration (minutes)'].median())
df['Rider wait time (minutes)'].fillna(df['Rider wait time (minutes)'].median())

3         3.3
5         1.0
12        8.8
22       13.3
32        4.7
         ... 
21306     7.4
21307    15.3
21314    18.9
21318     9.4
21319     1.3
Name: Rider wait time (minutes), Length: 2491, dtype: float64

In [24]:
df[features].isna().sum()

Delivery                     0
Distance                     0
Items in order               0
Bill subtotal                0
Total                        0
KPT duration (minutes)       0
Rider wait time (minutes)    0
Subzone                      0
City                         0
dtype: int64

In [25]:
df['high_rating'].value_counts()


high_rating
1    2088
0     403
Name: count, dtype: int64

In [26]:
x = df[features] 
y = df['high_rating'] 

In [27]:
x.head()

,Delivery,Distance,Items in order,Bill subtotal,Total,KPT duration (minutes),Rider wait time (minutes),Subzone,City
3,Zomato Delivery,2km,"1 x Fried Chicken Ghostbuster Tender, 1 x Anga...",620.0,567.00,19.00,3.3,Sector 4,Delhi NCR
5,Zomato Delivery,1km,"1 x Grilled Chicken Jamaican Tangdi, 1 x Bone ...",715.0,666.75,20.48,1.0,Sector 4,Delhi NCR
12,Zomato Delivery,3km,1 x Bone in Smoky Bbq Grilled Chicken,310.0,220.50,17.23,8.8,Sector 4,Delhi NCR
22,Zomato Delivery,5km,"1 x Animal Fries, 1 x Salted Fries",524.0,466.20,16.88,13.3,Sector 4,Delhi NCR
32,Zomato Delivery,3km,1 x Bone in Jamaican Grilled Chicken,695.0,645.75,7.87,4.7,Sector 4,Delhi NCR


In [29]:
y.value_counts()

high_rating
1    2088
0     403
Name: count, dtype: int64

In [32]:
df.loc[:, 'Items in order'] = df['Items in order'].apply(lambda x: len(str(x).split(',')))

In [34]:
df['Items in order'].head()

3     1
5     1
12    1
22    1
32    1
Name: Items in order, dtype: int64

In [35]:
x_encoded = pd.get_dummies(df[features], columns=['Delivery', 'City', 'Subzone'], drop_first=True)